# Movie Recommender System

## importing libraries

In [ ]:
import numpy as np 
import pandas as pd

## importing dataset

In [ ]:
movies = pd.read_csv('tmdb_5000_movie.csv')
credits= pd.read_csv('tmdb_5000_credit.csv')

## Date Preeprocessing

In [ ]:
movies.head()

In [ ]:
credits.head()

In [ ]:
credits.head()['cast'].values

In [ ]:
credits.head()['crew'].values

In [ ]:
movies =movies.merge(credits,on='title')

In [ ]:
movies.head(1)

In [ ]:
movies.info()

In [ ]:
# SELECTING THE IMPORTANT COLUMNS

# genre
# id
# keywords
# title
# overview
# cast 
# crew

movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [ ]:
movies.head()

In [ ]:
movies.isnull().sum()

In [ ]:
movies.dropna(inplace=True)

In [ ]:
movies.duplicated().sum()

In [ ]:
movies.iloc[0].genres

In [ ]:
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [ ]:
movies['genres']=movies['genres'].apply(convert)

In [ ]:
movies.head()

In [ ]:
movies['keywords']=movies['keywords'].apply(convert)

In [ ]:
movies.head(1)

In [ ]:
movies['cast'][0]

In [ ]:
def convert3(obj):
    L=[]
    counter =0
    for i in ast.literal_eval(obj):
        if counter != 3:
            L.append(i['name'])
            counter+=1
        else:
            break
    return L

In [ ]:
movies['cast']=movies['cast'].apply(convert3)

In [ ]:
movies['crew'][0]

In [ ]:
def fetch_director(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if i['job']== 'Director':
            L.append(i['name'])
            break
    return L

In [ ]:
movies['crew']=movies['crew'].apply(fetch_director)

In [ ]:
movies.head()

In [ ]:
movies['overview'][0]

In [ ]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [ ]:
movies.head()

In [ ]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","")for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","")for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","")for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","")for i in x])

In [ ]:
movies

In [ ]:
movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [ ]:
movies

In [ ]:
new_df= movies[['movie_id','title','tags']]

In [ ]:
new_df

In [ ]:
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))

In [ ]:
new_df

In [ ]:
new_df['tags']=new_df['tags'].apply(lambda x:x.lower())

In [ ]:
new_df.head()

In [ ]:
import nltk 
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(text):
    y =[]

    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
new_df['tags']=new_df['tags'].apply(stem)

## Vectorization

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv =CountVectorizer(max_features=5000,stop_words='english')
vectors =cv.fit_transform(new_df['tags']).toarray()

In [ ]:
vectors

In [ ]:
vectors[0]

In [ ]:
cv.get_feature_names_out()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity =cosine_similarity(vectors)

In [ ]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:6]

In [ ]:
similarity[0]  

## creating function

In [ ]:
def recommend(movie):
    movie_index=new_df[new_df['title']==movie].index[0]
    distances= similarity[movie_index]
    movie_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    for i in movie_list:
        print(new_df.iloc[i[0]].title)

In [ ]:
recommend('Batman Begins')

In [ ]:
import pickle

pickle.dump(new_df,open('movies.pkl','wb'))

In [ ]:
pickle.dump(new_df.to_dict(),open('movie_dict.pkl','wb'))

In [ ]:
pickle.dump(similarity,open ('similarity.pkl','wb'))